In [1]:
import imgaug as ia
from imgaug import augmenters as iaa
import cv2
import matplotlib.pyplot as plt
import numpy as np

def gen_matrix(image, nb_channels, random_state):
      matrix_A = np.array([[0, -1, 0],
                           [-1, 4, -1],
                           [0, -1, 0]])
      matrix_B = np.array([[0, 0, 0],
                          [0, -4, 1],
                          [0, 2, 1]])
      if random_state.rand() < 0.5:
          return [matrix_A] * nb_channels
      else:
          return [matrix_B] * nb_channels
# aug = iaa.Convolve(matrix=gen_matrix)

seq = iaa.Sequential([
    ## unsure if Crop is likely to be useful offsett may be better
#     iaa.Crop(px=(0, 16)), # crop images from each side by 0 to 16px (randomly chosen)
    ## definitely worth doing Fliplr as it doubles the size.
#     iaa.Fliplr(0.5), # horizontally flip 50% of the images
    ## GaussianBlur is also worth performing on for generalising
    ## possibly ignore it for my own data?  
#     iaa.GaussianBlur(sigma=(0, 3.0)), # blur images with a sigma of 0 to 3.0
    ## Noise is also worth doing for generalising.
#     iaa.AdditiveGaussianNoise(scale=0.1*255) #looks like static
#     iaa.EdgeDetect(alpha=(0.0, 1.0))
    
    ## I think convolve is probably useful if you define it with the sharpness to demonstrate
    ## the advantages of using Convolve is to view what the Computer might see.
    iaa.Convolve(matrix=gen_matrix),
    ## sharpeness definitely feels useful for defining the edges of the area.
#     iaa.Sharpen(alpha=(0.0, 1.0), lightness=(0.75, 2.0))
    ## probably worth doing sometimes, it might be better with 
#     iaa.ContrastNormalization((0.5, 1.5))
#     iaa.ContrastNormalization((0.5, 1.5), per_channel=0.5),
    ## Affine scale
    ## might be an issue due to where the signs are typically located don't increase the sign beyond 1.0
    ## unless you write a function to discard bounding boxes outside the actual area.
#     iaa.Affine(scale=(0.35, .90)),
    ## scale indepently on each axis indepently changing axis, might be really useful
#     iaa.Affine(scale={'x': (0.5, 1), 'y': (0.35, 1.0)}),
    ## this could be useful however, it would require it the translation to not cut off the signs in the images
    ## possible function to ensure it's not missing
    ## note though it's on percent
#     iaa.Affine(translate_percent={'x': (-0.02, 0.2), 'y': (-0.2, 0.2)})
    ## note the below is a pixel translation which might actually be better due to it being more minor
#     iaa.Affine(translate_pixel={'x': (-20, 20), 'y': (-20, 20)})
    ## note that this will be worth while if it's using with scaling and/or doesn't cut off
    ## the actual sign
#     iaa.Affine(rotate=(-45, 45))
    ## shear is definitely worth including in the data augmentation process
#     iaa.Affine(shear=(-160,160))
    ## unsure of what the mode kwarg is doing currently
#     iaa.Affine(translate_percent={"x": (-0.20)}, mode=ia.ALL, cval=(0, 255))
    ## Really useful on low level distortion
#     iaa.PiecewiseAffine(scale=(0.008, 0.0095))
    ## use ElasticTransformation to add random can pass in tuples for randomness
    ## to the distortion field
    iaa.ElasticTransformation(alpha=(0, 5.0), sigma=(0.25, 1))

])
img1 = cv2.imread('/Users/datascience4/Documents/highways-deep-learning/data/A3400_NB1_RAV_R07_180516084515_frame_235.jpg')

img = cv2.imread('/Users/datascience4/Documents/highways-deep-learning/data/A3400_NB1_RAV_R07_180516084515_frame_105.jpg')
# cv2.imshow('test', img)
imgs = [img, img1]
print(type(img))
# seq.show_grid(imgs[0], cols=8, rows=8)
# cropped = img[7:100, 22:67]
# # cv2.imshow('cropped', cropped)
# # cv2.waitKey(0)

img_aug = seq.augment_image(img)
# seq.show_grid(img_aug, cols=8, rows=8)
# print(img.shape)
# print(img_aug.shape)
# print(len(img_aug))

# cv2.imshow('aug??', img_aug)
# cv2.waitKey(0)

# # for batch_idx in range(1000):
# #     # 'images' should be either a 4D numpy array of shape (N, height, width, channels)
# #     # or a list of 3D numpy arrays, each having shape (height, width, channels).
# #     # Grayscale images must have shape (height, width, 1) each.
# #     # All images must have numpy's dtype uint8. Values are expected to be in
# #     # range 0-255.
# #     images = load_batch(batch_idx)
# #     images_aug = seq.augment_images(images)
# #     train_on_images(images_aug)

<class 'numpy.ndarray'>


# Write out one image as new.

1. It needs the path of the image.
2. The bounding boxes of the image.
3. Write the number 

In [2]:
import os
import xml.etree.ElementTree as ET
import cv2

base_dir = '/Users/datascience4/Documents/training3'
print(len(os.listdir(base_dir)))
xml_files = os.listdir(base_dir)
xml_files = [x for x in os.listdir(base_dir) if '.xml' in x]
print(xml_files)
items = []

## come back to this

#let's read the first xml
test_xml = os.path.join(base_dir, xml_files[0])
tree = ET.parse(test_xml)
root = tree.getroot()
## path can be used for cv2 read image
path = root.find('path').text
print(path)
## find the object and xmin, ymin, xmax, ymax
for child in root.iter('object'):
    if child.find('bndbox'):
        bndbox = child.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        
print(xmin, ymin, xmax, ymax)

# ia.seed(1)
## pass the path into cv2
image = cv2.imread(path)
# cv2.imshow('path', image)
# cv2.waitKey(0)

## bounding boxes now for the image takes a list
bbs = ia.BoundingBoxesOnImage([
    ia.BoundingBox(x1=xmin, y1=ymin, x2=xmax, y2=ymax)
], shape=image.shape)

seq = iaa.Sequential([
    iaa.Multiply((1.2, 1.5)), # change brightness, doesn't affect bb
    iaa.Affine(
        translate_px={"x": 40, "y": 60},
#         scale=(0.5, 0.7),
        rotate=(-25, 25)
        
    ) # translate by 40/60px on x/y axis, and scale to 50-70%, affects BBs
])

seq = iaa.Sequential([
    iaa.Fliplr(0.5), #horizontal flips
    iaa.Crop(percent=(0, 0.1)), # random crops
    #Small gaussian blur with random sigma between 0 and 0.5
    # But we only blur about 50% of all images.
    iaa.Sometimes(0.5,
                 iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    #Strengthen or weaken the contrast in each image.
    iaa.ContrastNormalization((0.75, 1.5)),
    #Add gaussian noise.
    #For 50% of all images, we sample the noise once per pixel..
    #For the other 50% of all images, we sample the noise per pixel
    #AND channel. This can change the color (not only the brightness)
    #of the pixels.
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255),
                             per_channel=0.5),
    #Make some images brighter and some darker.
    #in 20% of all cases we sample the multiplier once per channel.
    # which can end up changing the color of the images.
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    #Apply affine transformations to each image.
    #Scale/zoom them, translate/move them, rotate them, and shear them.
    iaa.Affine(
        scale={'x': (0.8, 1.2), 'y': (0.8, 1.2)},
        translate_percent={'x': (-0.2, 0.2), 'y': (-0.2, 0.2)},
        rotate=(-25, 25),
        shear=(-8, 8)
    )
], random_order=True) #apply augmenters in random order

## make our sequence deterministic.
# we can now apply it to the image and then to the bbs and it will
# lead to the same augmentations
# IMPORTANT: call this once PER BATCH, otherwise you will always
# get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

##Augment bbs and images.
## as we only have one image and list of bbs we use
## [image] and [bbs] to turn both into lists (batches) for the
## functions and then [0] to reverse that. In a real experiment, your
## variables would likely already be lists.
image_aug = seq_det.augment_images([image])[0]
bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

##print the coordinates before/after augmentation(see below)
## use .x1_int, y1_int, ... to get interger coordinates
for i in range(len(bbs.bounding_boxes)):
    before = bbs.bounding_boxes[i]
    after = bbs_aug.bounding_boxes[i]
    print(f"BB {i}:, ({before.x1, before.y1, before.x2, before.y2} \
          -> ({after.x1, after.y1, after.x2, after.y2})")
    
# image with bbs before/after augmentation (shown below)
image_before = bbs.draw_on_image(image, thickness=2)
# cv2.imshow('image before', image_before)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

image_after = bbs_aug.draw_on_image(image_aug, thickness=2, color=[0, 0, 255])
# cv2.imshow('image after', image_after)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

cv2.imwrite('/Users/datascience4/Desktop/test1.png', image_after)

385
['A3400_NB1_RAV_R07_180516084515_frame_3598.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2862.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2447.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2335.xml', 'A3400_NB1_RAV_R07_180516084515_frame_4022.xml', 'A3400_NB1_RAV_R07_180516084515_frame_224.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2255.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2527.xml', 'A3400_NB1_RAV_R07_180516084515_frame_3163.xml', 'A3400_NB1_RAV_R07_180516084515_frame_350.xml', 'A3400_NB1_RAV_R07_180516084515_frame_3375.xml', 'A3400_NB1_RAV_R07_180516084515_frame_3374.xml', 'A3400_NB1_RAV_R07_180516084515_frame_3837.xml', 'A3400_NB1_RAV_R07_180516084515_frame_3162.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2240.xml', 'A3400_NB1_RAV_R07_180516084515_frame_225.xml', 'A3400_NB1_RAV_R07_180516084515_frame_4023.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2446.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2863.xml', 'A3400_NB1_RAV_R07_180516084515_frame_3599.xml', 'A3400_NB1_RAV_R07

True

## Things to do
1. Create a folder for the augmented images
2. Then run through all images (drawing on each image the bounding box)
3. Sense check that all are in the bounding boxes.

In [3]:
try:
    os.mkdir('/Users/datascience4/Documents/training_3_data_aug')
except FileExistsError:
    print('folder already exists')

folder already exists


In [6]:
aug_dir = '/Users/datascience4/Documents/training_3_data_aug'

## SECTION DEALS WITH BOUNDING BOXES NOT IN THE IMAGE
## funtion to pad the image and deal with bounding boxes not in the images

# Pad image with a 1px white and (BY-1)px black border
def pad(image, by):
    if by <= 0:
        return image
    image_border1 = np.pad(
        image, ((1, 1), (1, 1), (0, 0)),
        mode="constant", constant_values=255
    )
    image_border2 = np.pad(
        image_border1, ((by-1, by-1), (by-1, by-1), (0, 0)),
        mode="constant", constant_values=0
    )
    return image_border2

# Draw BBs on an image
# and before doing that, extend the image plane by BORDER pixels.
# Mark BBs inside the image plane with green color, those partially inside
# with orange and those fully outside with red.
def draw_bbs(image, bbs, border):
    ##rgb in open cv isn't rgb but bgr - it's bad for that!
    GREEN = (0, 255, 0)
    ORANGE = (0, 140, 255)
    RED = (0, 0, 255)
    image_border = pad(image, border)
    for bb in bbs.bounding_boxes:
        if bb.is_fully_within_image(image.shape):
            color = GREEN
        elif bb.is_partly_within_image(image.shape):
            print('partly?')
            color = ORANGE
        else:
            print('not in at all')
            color = RED
        image_border = bb.shift(left=border, top=border).draw_on_image(image_border, thickness=2, color=color)
    return image_border

def draw_bbs_count(image, bbs, border, **kwargs):
    counter = kwargs.get('counter', None)
    ## using open CV2 so colours are in BGR and not RGB so check the order if it comes different
    GREEN = (0, 255, 0)
    ORANGE = (0, 140, 255)
    RED = (0, 0, 255)
    ## probably can remove the ine beloe as it won't be needed
    image_border = pad(image, border)
    for bb in bbs.bounding_boxes:
        if bb.is_fully_within_image(image.shape):
            color = GREEN
            counter += 1
        elif bb.is_partly_within_image(image.shape):
            print('partly version 2')
            color = ORANGE
            image_border = None
        else:
            print('not in at all')
            color = RED
            image_border = None
        if image_border is not None:
            image_border = bb.shift(left=border, top=border).draw_on_image(image_border, thickness=2, color=color)
    return image_border, counter
    
    

## write out a file that makes ue of similar layout as draw_bbs



## attempt to write out the same image multiple times in different ways
for i in range(10):
    #let's read the first xml
    test_xml = os.path.join(base_dir, xml_files[i])
    print(i)
    tree = ET.parse(test_xml)
    root = tree.getroot()
    ## path can be used for cv2 read image
    path = root.find('path').text
    print(path)
    ## find the object and xmin, ymin, xmax, ymax
    for child in root.iter('object'):
        if child.find('bndbox'):
            bndbox = child.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
    
    image = cv2.imread(path)
    bbs = ia.BoundingBoxesOnImage([
                    ia.BoundingBox(x1=xmin, y1=ymin, x2=xmax, y2=ymax)
    ], shape=image.shape)
    ## new
    counter = 0
    ## Set the counter to less than the amount of images you actually want, so if it's 50 of each individual image
    while counter < 20:
#         counter += 1
        print(counter)
        seq_det = seq.to_deterministic()
        image_aug = seq_det.augment_images([image])[0]
        bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]
        image_after, counter = draw_bbs_count(image_aug, bbs_aug, 1, counter=counter)
        if image_after is not None:
            cv2.imwrite(os.path.join(aug_dir, ('bb_count_photo' + str(i) + 'counter_no' + str(counter) + '.png')), image_after)
        print(bbs, bbs_aug)
        
    ##
#     for j in range(10):
#         seq_det = seq.to_deterministic()
#         image_aug = seq_det.augment_images([image])[0]
#         bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]
#         ## test removing
#         image_after = bbs_aug.draw_on_image(image_aug, thickness=2, color=[255, 140, 0])
#         ## testing
#         image_after1 = draw_bbs(image_aug, bbs_aug, 100, j)
#         cv2.imwrite(os.path.join(aug_dir, ('bb' + str(i) + str(j) + '.png')), image_after1)
# #         image_after2 = draw_bbs(image_aug, bbs_aug.remove_out_of_image(), 100)
# #         cv2.imwrite(os.path.join(aug_dir, ('bb_removed' + str(i) + str(j) + '.png')), image_after2)
# #         image_after3 = draw_bbs(image_aug, bbs_aug.remove_out_of_image()\
# #                 .cut_out_of_image(), 100)
# #         cv2.imwrite(os.path.join(aug_dir, ('bb_removed_and_cut_out' + str(i) + str(j) + '.png')), image_after3)
# #         cv2.imwrite(os.path.join(aug_dir, (str(i) + str(j) + '.png')), image_after)
#         print(bbs, bbs_aug)


0
/Users/datascience4/Documents/highways-deep-learning/data/A3400_NB1_RAV_R07_180516084515_frame_3598.jpg
0
the bb BoundingBox(x1=520.4125, y1=75.7854, x2=575.7868, y2=121.7193, label=None)
BoundingBoxesOnImage([BoundingBox(x1=737.0000, y1=204.0000, x2=793.0000, y2=250.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=520.4125, y1=75.7854, x2=575.7868, y2=121.7193, label=None)], shape=(768, 1360, 3))
1
the bb BoundingBox(x1=765.5112, y1=151.8334, x2=836.3927, y2=202.5372, label=None)
BoundingBoxesOnImage([BoundingBox(x1=737.0000, y1=204.0000, x2=793.0000, y2=250.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=765.5112, y1=151.8334, x2=836.3927, y2=202.5372, label=None)], shape=(768, 1360, 3))
2
the bb BoundingBox(x1=893.7875, y1=261.2144, x2=969.7816, y2=335.8614, label=None)
BoundingBoxesOnImage([BoundingBox(x1=737.0000, y1=204.0000, x2=793.0000, y2=250.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingB

the bb BoundingBox(x1=932.2402, y1=37.1016, x2=970.3631, y2=74.9966, label=None)
BoundingBoxesOnImage([BoundingBox(x1=845.0000, y1=200.0000, x2=873.0000, y2=224.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=932.2402, y1=37.1016, x2=970.3631, y2=74.9966, label=None)], shape=(768, 1360, 3))
6
the bb BoundingBox(x1=543.6387, y1=280.9682, x2=579.2935, y2=318.2306, label=None)
BoundingBoxesOnImage([BoundingBox(x1=845.0000, y1=200.0000, x2=873.0000, y2=224.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=543.6387, y1=280.9682, x2=579.2935, y2=318.2306, label=None)], shape=(768, 1360, 3))
7
the bb BoundingBox(x1=813.4941, y1=320.5614, x2=844.8044, y2=348.9252, label=None)
BoundingBoxesOnImage([BoundingBox(x1=845.0000, y1=200.0000, x2=873.0000, y2=224.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=813.4941, y1=320.5614, x2=844.8044, y2=348.9252, label=None)], shape=(768, 1360, 3))
8
the bb BoundingBo

BoundingBoxesOnImage([BoundingBox(x1=322.0000, y1=124.0000, x2=362.0000, y2=181.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=135.6904, y1=361.3167, x2=213.6509, y2=438.3061, label=None)], shape=(768, 1360, 3))
8
not in at all
BoundingBoxesOnImage([BoundingBox(x1=322.0000, y1=124.0000, x2=362.0000, y2=181.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=1367.8763, y1=237.5419, x2=1444.9824, y2=310.0180, label=None)], shape=(768, 1360, 3))
8
the bb BoundingBox(x1=516.0148, y1=3.5665, x2=579.6075, y2=72.9861, label=None)
BoundingBoxesOnImage([BoundingBox(x1=322.0000, y1=124.0000, x2=362.0000, y2=181.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=516.0148, y1=3.5665, x2=579.6075, y2=72.9861, label=None)], shape=(768, 1360, 3))
9
the bb BoundingBox(x1=377.4214, y1=109.6959, x2=427.9835, y2=182.0278, label=None)
BoundingBoxesOnImage([BoundingBox(x1=322.0000, y1=124.0000, x2=362.0000, y2=181.0000, 

partly version 2
BoundingBoxesOnImage([BoundingBox(x1=245.0000, y1=173.0000, x2=287.0000, y2=208.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=337.6682, y1=-36.4746, x2=393.9716, y2=16.5791, label=None)], shape=(768, 1360, 3))
7
the bb BoundingBox(x1=881.5531, y1=40.4873, x2=937.2366, y2=80.3038, label=None)
BoundingBoxesOnImage([BoundingBox(x1=245.0000, y1=173.0000, x2=287.0000, y2=208.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=881.5531, y1=40.4873, x2=937.2366, y2=80.3038, label=None)], shape=(768, 1360, 3))
8
partly version 2
BoundingBoxesOnImage([BoundingBox(x1=245.0000, y1=173.0000, x2=287.0000, y2=208.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=-18.0734, y1=-27.4548, x2=29.5542, y2=22.7235, label=None)], shape=(768, 1360, 3))
8
the bb BoundingBox(x1=206.5369, y1=218.8875, x2=246.9220, y2=256.8848, label=None)
BoundingBoxesOnImage([BoundingBox(x1=245.0000, y1=173.0000, x2=287.00

the bb BoundingBox(x1=150.3681, y1=219.5791, x2=481.5713, y2=387.6416, label=None)
BoundingBoxesOnImage([BoundingBox(x1=898.0000, y1=188.0000, x2=1143.0000, y2=286.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=150.3681, y1=219.5791, x2=481.5713, y2=387.6416, label=None)], shape=(768, 1360, 3))
6
the bb BoundingBox(x1=212.5455, y1=125.7637, x2=462.0429, y2=291.3872, label=None)
BoundingBoxesOnImage([BoundingBox(x1=898.0000, y1=188.0000, x2=1143.0000, y2=286.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=212.5455, y1=125.7637, x2=462.0429, y2=291.3872, label=None)], shape=(768, 1360, 3))
7
partly version 2
BoundingBoxesOnImage([BoundingBox(x1=898.0000, y1=188.0000, x2=1143.0000, y2=286.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=252.8945, y1=-73.2841, x2=520.0467, y2=106.5010, label=None)], shape=(768, 1360, 3))
7
the bb BoundingBox(x1=197.4460, y1=152.0900, x2=464.2152, y2=328.7992, label

not in at all
BoundingBoxesOnImage([BoundingBox(x1=1069.0000, y1=69.0000, x2=1108.0000, y2=147.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=-181.4507, y1=129.0399, x2=-119.7498, y2=236.8936, label=None)], shape=(768, 1360, 3))
1
the bb BoundingBox(x1=1277.3280, y1=63.7673, x2=1356.3089, y2=159.9641, label=None)
BoundingBoxesOnImage([BoundingBox(x1=1069.0000, y1=69.0000, x2=1108.0000, y2=147.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=1277.3280, y1=63.7673, x2=1356.3089, y2=159.9641, label=None)], shape=(768, 1360, 3))
2
the bb BoundingBox(x1=946.8953, y1=73.0183, x2=1013.6418, y2=176.7740, label=None)
BoundingBoxesOnImage([BoundingBox(x1=1069.0000, y1=69.0000, x2=1108.0000, y2=147.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=946.8953, y1=73.0183, x2=1013.6418, y2=176.7740, label=None)], shape=(768, 1360, 3))
3
partly version 2
BoundingBoxesOnImage([BoundingBox(x1=1069.0000, y1=69.0000

the bb BoundingBox(x1=47.0474, y1=30.1534, x2=109.3968, y2=129.9406, label=None)
BoundingBoxesOnImage([BoundingBox(x1=1069.0000, y1=69.0000, x2=1108.0000, y2=147.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=47.0474, y1=30.1534, x2=109.3968, y2=129.9406, label=None)], shape=(768, 1360, 3))
17
the bb BoundingBox(x1=915.4684, y1=102.6517, x2=975.5751, y2=208.5700, label=None)
BoundingBoxesOnImage([BoundingBox(x1=1069.0000, y1=69.0000, x2=1108.0000, y2=147.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=915.4684, y1=102.6517, x2=975.5751, y2=208.5700, label=None)], shape=(768, 1360, 3))
18
not in at all
BoundingBoxesOnImage([BoundingBox(x1=1069.0000, y1=69.0000, x2=1108.0000, y2=147.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=1282.2869, y1=-102.4246, x2=1338.3007, y2=-29.7047, label=None)], shape=(768, 1360, 3))
18
not in at all
BoundingBoxesOnImage([BoundingBox(x1=1069.0000, y1=69.0000, x2=

the bb BoundingBox(x1=620.4019, y1=99.0524, x2=672.9550, y2=157.2684, label=None)
BoundingBoxesOnImage([BoundingBox(x1=408.0000, y1=95.0000, x2=452.0000, y2=137.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=620.4019, y1=99.0524, x2=672.9550, y2=157.2684, label=None)], shape=(768, 1360, 3))
19
the bb BoundingBox(x1=3.2671, y1=155.2017, x2=60.6065, y2=214.9806, label=None)
BoundingBoxesOnImage([BoundingBox(x1=408.0000, y1=95.0000, x2=452.0000, y2=137.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=3.2671, y1=155.2017, x2=60.6065, y2=214.9806, label=None)], shape=(768, 1360, 3))
7
/Users/datascience4/Documents/highways-deep-learning/data/A3400_NB1_RAV_R07_180516084515_frame_2527.jpg
0
the bb BoundingBox(x1=1195.0611, y1=187.1830, x2=1231.1771, y2=215.4838, label=None)
BoundingBoxesOnImage([BoundingBox(x1=241.0000, y1=117.0000, x2=271.0000, y2=145.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=1

BoundingBoxesOnImage([BoundingBox(x1=241.0000, y1=117.0000, x2=271.0000, y2=145.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=65.2428, y1=177.6825, x2=106.9846, y2=220.9041, label=None)], shape=(768, 1360, 3))
14
the bb BoundingBox(x1=27.3295, y1=28.9719, x2=73.6282, y2=74.1395, label=None)
BoundingBoxesOnImage([BoundingBox(x1=241.0000, y1=117.0000, x2=271.0000, y2=145.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=27.3295, y1=28.9719, x2=73.6282, y2=74.1395, label=None)], shape=(768, 1360, 3))
15
not in at all
BoundingBoxesOnImage([BoundingBox(x1=241.0000, y1=117.0000, x2=271.0000, y2=145.0000, label=None)], shape=(768, 1360, 3)) BoundingBoxesOnImage([BoundingBox(x1=1169.5194, y1=-49.2755, x2=1213.5433, y2=-10.6998, label=None)], shape=(768, 1360, 3))
15
the bb BoundingBox(x1=935.1449, y1=108.7403, x2=972.5675, y2=138.4212, label=None)
BoundingBoxesOnImage([BoundingBox(x1=241.0000, y1=117.0000, x2=271.0000, y2=145.0000, 

KeyboardInterrupt: 

# The above works but needs improvement
Things now required.
* Bounding boxes are now getting detected inside each image, as well as in the augmented images whether or not they exist in boundaries of that image using the function draw_bbs with padding.
* Even on minor partial images we now need to cut them and redo the augmentations.
* Write a function that only write an xml file and bounding box coordinates if the are not just partially in but fully in.
* If partial or out the augmentation needs to go again until randomly generating one with bounding boxes inside the image. Otherwise it will be hinder the model.
* A function similar to draw_bbs should be able to accomplish this.

* Bounding boxes need to be in the image, if they aren't they should be disgarded and a new image created that until the bounding box sits inside the image and can then be counted.


* Address the category imbalance as well with this work. That if takes the classes, and counts them, that it should then run each for the x amount of classes. related to each type of work.

In [ ]:
print(bbs.bounding_boxes)
for bb in bbs.bounding_boxes:
    print(bb)

In [ ]:
image.shape
